<a href="https://colab.research.google.com/github/Aravindh4404/FYPSeagullClassification01/blob/main/OrigAdultInflightResnet1410.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Mount Google Drive to access/save files
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# Define paths to training data
train_data_root = "/content/drive/My Drive/FYP/Dataset/Original_Adult_In-flight/train"
test_data_root = "/content/drive/My Drive/FYP/Dataset/Original_Adult_In-flight/test"
IMAGE_SHAPE = (224, 224)  # Image size for ResNet50
BATCH_SIZE = 32  # Adjust batch size as per the dataset size
EPOCHS = 20  # Set the number of epochs

# Output paths for saving models and class indices
MODEL_SAVE_PATH = '/content/drive/My Drive/FYP/MODELS/best_resnet50_model.h5'
FINAL_MODEL_SAVE_PATH = '/content/drive/My Drive/FYP/MODELS/resnet50_final_model.h5'
CLASS_INDICES_SAVE_PATH = '/content/drive/My Drive/FYP/class_indices.npy'

# Create ImageDataGenerator for training and validation with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the pixel values
    validation_split=0.3,  # 30% of data used for validation
    rotation_range=10,  # Augmentation: rotate images by 10 degrees
    zoom_range=0.1,  # Augmentation: zoom in by 10%
    horizontal_flip=True  # Augmentation: randomly flip images horizontally
)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_data_root,
    subset="training",
    target_size=IMAGE_SHAPE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Validation generator
valid_generator = train_datagen.flow_from_directory(
    train_data_root,
    subset="validation",
    target_size=IMAGE_SHAPE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Save class indices mapping for future use during testing
class_indices = train_generator.class_indices
np.save(CLASS_INDICES_SAVE_PATH, class_indices)
print(f"Class indices saved: {class_indices}")

# Load pre-trained ResNet50 model, excluding the top layers
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=IMAGE_SHAPE + (3,))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global pooling layer
x = Dense(1024, activation='relu')(x)  # Fully connected layer with 1024 units
predictions = Dense(len(class_indices), activation='softmax')(x)  # Output layer for classification

# Define the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the ResNet50 base layers initially (only train new layers)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("Model compiled successfully.")

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# Update model save path to use `.keras` extension
MODEL_SAVE_PATH = '/content/drive/My Drive/FYP/MODELS/best_resnet50_model.keras'
FINAL_MODEL_SAVE_PATH = '/content/drive/My Drive/FYP/MODELS/resnet50_final_model.keras'

# Model checkpoint to save the best model based on validation loss
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_loss', mode='min')

# List of callbacks
callbacks = [early_stopping, model_checkpoint]

# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)

# Save the final model after training using the `.keras` format
model.save(FINAL_MODEL_SAVE_PATH)
print(f"Final model saved at: {FINAL_MODEL_SAVE_PATH}")

# Unfreeze some of the base model layers for fine-tuning
for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers
    layer.trainable = True

# Re-compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,  # Fine-tune for a few epochs
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=valid_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)

# Save the fine-tuned model
fine_tuned_model_path = '/content/drive/My Drive/FYP/MODELS/resnet50_fine_tuned_model.h5'
model.save(fine_tuned_model_path)
print(f"Fine-tuned model saved at: {fine_tuned_model_path}")

# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(valid_generator, steps=valid_generator.samples // BATCH_SIZE)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 176 images belonging to 2 classes.
Found 75 images belonging to 2 classes.
Class indices saved: {'Glaucous_Winged_Gull': 0, 'Slaty_Backed_Gull': 1}
Model compiled successfully.
Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.5378 - loss: 1.3154 - val_accuracy: 0.4219 - val_loss: 1.0248
Epoch 2/20
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.5312 - loss: 0.8491

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 731ms/step - accuracy: 0.5312 - loss: 0.8491 - val_accuracy: 0.0000e+00 - val_loss: 1.2134
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 533ms/step - accuracy: 0.4606 - loss: 0.9519 - val_accuracy: 0.4219 - val_loss: 1.0701
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 455ms/step - accuracy: 0.4062 - loss: 1.0920 - val_accuracy: 1.0000 - val_loss: 0.2076
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 382ms/step - accuracy: 0.5681 - loss: 0.7576 - val_accuracy: 0.5781 - val_loss: 0.7509
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 330ms/step - accuracy: 0.5000 - loss: 0.8533 - val_accuracy: 0.0000e+00 - val_loss: 0.9862
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 368ms/step - accuracy: 0.5660 - loss: 0.6838 - val_accuracy: 0.4219 - val_loss: 0.7590
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.4688 - loss: 0.6779 - val_accuracy: 0.6364 - val_loss: 0.6717
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 502ms/step - accuracy: 0.5821 - loss: 0.6802 - val_accuracy: 0.4219 - val_loss: 0.7

Fine-tuned model saved at: /content/drive/My Drive/FYP/MODELS/resnet50_fine_tuned_model.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step - accuracy: 0.2812 - loss: 1.1431
Validation Loss: 0.9727997779846191
Validation Accuracy: 0.421875
